In [ ]:
#from langchain_community.llms import Ollama # old
from langchain_ollama import ChatOllama # new
from langchain_ollama import OllamaLLM
#from langchain_community.vectorstores import Chroma # old
from langchain_chroma import Chroma
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain.chains import RetrievalQA

import os

- Initiate VectorDB

In [ ]:
# Initialize the embedding
# oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
oembed = OllamaEmbeddings(model="llama3.2:1b")


In [ ]:
# Connect to existing vectorstore
vectorstore = Chroma(
    collection_name="address_metadata",
    embedding_function=oembed,
    persist_directory="./vectorstore"
)

- Initiate LLM

In [ ]:
# Initialize the LLM
#llm = Ollama(model="deepseek-r1:14b", base_url="http://localhost:11434")


# Initialize the LLM
llm = OllamaLLM(model="deepseek-r1:14b", base_url="http://localhost:11434")
# llm = ChatOllama(model="deepseek-r1:14b", base_url="http://localhost:11434")


In [ ]:
llm.invoke("Hello world")

- RAG retrievers

In [ ]:
import chromadb

# Initialize the persistent ChromaDB client
# chroma_client = chromadb.PersistentClient(path="./vectorstore")
chroma_client = chromadb.PersistentClient(path="https://drive.google.com/drive/folders/1tl1V-lsDYSt5GFILaKowd0pDM00Z5DX7")

# List all collections in the database
collections = chroma_client.list_collections()
print("Collections in the database:")
for collection in collections:
    # Print the collection name directly since collection is now a string
    print(collection)

# If you need to work with a specific collection, use get_collection():
# Example:
# collection = chroma_client.get_collection(name="your_collection_name")

address = chroma_client.get_collection(name="address_metadata")

# show the number of documents in the collection
print(address.count())

In [ ]:
import pandas as pd
state = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["state"]].drop_duplicates()['state'].str.lower().tolist()
district = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["district"]].drop_duplicates()['district'].str.lower().tolist()
mukim = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["mukim"]].dropna().drop_duplicates()['mukim'].str.lower().tolist()
location = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()

postcode = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv",dtype='str')[["postcode"]].dropna().drop_duplicates()['postcode'].str.lower().tolist()
postcode = [x.strip() for x in list(set(postcode)) for x in x.split(",")]



In [ ]:
input_str = "235, pintasan mayang 1, persiaran mayang pasir, 11900, mukim 12, pulau pinang"

# find the state, district, mukim and location in the string
state_found = [s for s in state if s in input_str.lower()] # case len = 1 , case len = 0 , case len > 1
district_found = [d for d in district if d in input_str.lower()]
mukim_found = [m for m in mukim if m in input_str.lower()]
location_found = [l for l in location if l in input_str.lower()]  # Convert float to string

postcode_found = [p for p in postcode if p in input_str.lower()]


In [ ]:
print(state_found)
print(district_found)  
print(mukim_found)
print(location_found)

print(postcode_found)

In [ ]:
'''
1)state_found --|-->--- 2)district_found --|-->---- 4)mukim_found --|--->--- 8)location_found
                |                          |                        |--->--- 9)not location_found
                |                          |    
                |                          |--> 5)not mukim_found --|--->--- 10)location_found
                |                                                   |--->--- 11)not location_found
                |
                |
                |--> 3)no district_found --|-->---- 6)mukim_found --|--->--- 12)location_found
                                           |                        |--->--- 13)not location_found
                                           |                      
                                           |--> 7)not mukim_found --|--->--- 14)location_found   
                                                                    |--->--- 15)not location_found
'''
'''             |                          |                        |
    Level 1     |               Level 2    |             Level 3    |             Level 4   
                |                          |                        |              
'''

In [ ]:
state_update = []
district_update = []
mukim_update = []
location_update = []


str = "235, pintasan mayang 1, persiaran mayang pasir, 11900, bayan lepas, pulau pinang"

In [ ]:
# level 1,2,3,4

import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

state = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["state"]].drop_duplicates()['state'].str.lower().tolist()
district = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["district"]].drop_duplicates()['district'].str.lower().tolist()
mukim = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["mukim"]].dropna().drop_duplicates()['mukim'].str.lower().tolist()
location = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()

# find the state, district, mukim and location in the string
state_found = [s for s in state if s in str.lower()] # case len = 1 , case len = 0 , case len > 1
district_found = [d for d in district if d in str.lower()]
mukim_found = [m for m in mukim if m in str.lower()]
location_found = [l for l in location if l in str.lower()]  # Convert float to string

# 1)
if len(state_found) == 1:                                                                                                                 
    print("State found:", state_found[0])

    state_update=state_update+(state_found)

    state = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")
    
    #filter state_found
    state = state[state['state'].str.lower() == state_found[0]]

    #update district_found
    district = state[["district"]].drop_duplicates()['district'].str.lower().tolist()
    district_found = [d for d in district if d in str.lower()]

    ####################################################################################################################

    # 2)
    if district_found:
                                                                                                                                                                                             
        for district in district_found:

            # district name is not the same as state name
            # 2a)
            if district != state_found[0]:                                                                                                   
                print(f"District found for the state {state_found[0]}:", district)
                district_update=district_update+([district])
                # filter the district
                district_case1 = state[state['district'].str.lower() == district]
                # update mukim_found
                mukim = district_case1[["mukim"]].dropna().drop_duplicates()['mukim'].str.lower().tolist()
                mukim_found = [m for m in mukim if m in str.lower()]

            ##########################################################################################################

                # 4)
                if mukim_found:
                    for mukim in mukim_found:
                        # mukim name is not the same as district name
                        if mukim != district:
                            print("Mukim name found not same as district name:", mukim)
                            mukim_update=mukim_update+([mukim])
                            # filter the mukim
                            mukim_case1 = district_case1[district_case1['mukim'].str.lower() == mukim]
                            # update location_found
                            location = mukim_case1[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
                            #location_found = [l for l in location if l in str.lower()]
                            location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
                            # 8) [1248]
                            if location_found:
                                for location in location_found:
                                    if location != mukim_found[0]:
                                        print(f"Location found for the mukim {mukim}:", location)
                                        location_update=location_update+([location])
                        # mukim name is the same as district name
                        elif mukim == district:
                            print("Mukim name found same as district name")
                            mukim_update=mukim_update+([mukim])
                            # filter the mukim
                            mukim_case2 = district_case1[district_case1['mukim'].str.lower() == mukim]
                            # update location_found
                            # 8) [1248]
                            location = mukim_case2[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
                            #location_found = [l for l in location if l in str.lower()]
                            location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
                            print(f"Location found for the mukim {mukim}:", location)
                            location_update=location_update+([location])

            ##########################################################################################################

                # 5)    
                elif not mukim_found:
                    print("No Mukim found for the district:", district)                    
                    # update location_found
                    location = district_case1[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
                    #location_found = [l for l in location if l in str.lower()]
                    location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
                    # 10) [12510]
                    if location_found:
                        for location in location_found:
                            if location != district:
                                print(f"Location found for the district {district}:", location)
                                location_update=location_update+([location])
            
            ##########################################################################################################
            
            # district name is the same as state name
            # 2b)
            elif district == state_found[0]:

                print(f"District found for the state {state_found[0]}:", district)
                district_update=district_update+([district])

                # filter the district
                district_case2 = state[state['district'].str.lower() == district]
                
                # update mukim_found
                mukim = district_case2[["mukim"]].dropna().drop_duplicates()['mukim'].str.lower().tolist()
                mukim_found = [l for l in mukim if l in str.lower()]

            ##########################################################################################################
            
                # 4)
                if mukim_found:
                    for mukim in mukim_found:
                        # mukim name is not the same as district name
                        if mukim != district:
                            print("Mukim name found not same as district name:", mukim)
                            mukim_update=mukim_update+([mukim])
                            # filter the mukim
                            mukim_case1 = district_case2[district_case2['mukim'].str.lower() == mukim]
                            # update location_found
                            location = mukim_case1[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
                            #location_found = [l for l in location if l in str.lower()]
                            location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
                            # 8) [1248]
                            if location_found:
                                for location in location_found:
                                    if location != mukim_found[0]:
                                        print(f"Location found for the mukim {mukim}:", location)
                                        location_update=location_update+([location])
                        # mukim name is the same as district name
                        elif mukim == district:
                            print("Mukim name found same as district name")
                            mukim_update=mukim_update+([mukim])
                            # filter the mukim
                            mukim_case2 = district_case2[district_case2['mukim'].str.lower() == mukim]
                            # update location_found
                            # 8) [1248]
                            location = mukim_case2[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
                            #location_found = [l for l in location if l in str.lower()]
                            location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
                            print(f"Location found for the mukim {mukim}:", location)
                            location_update=location_update+([location])


            ##########################################################################################################

                # 5)    
                elif not mukim_found:
                    print("No Mukim found for the district:", district)                    
                    # update location_found
                    location = district_case2[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
                    #location_found = [l for l in location if l in str.lower()]
                    location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
                    # 10) [12510]
                    if location_found:
                        for location in location_found:
                            if location != district:
                                print(f"Location found for the district {district}:", location)
                                location_update=location_update+([location])
print()
print("state_update:", state_update)
print("district_update:", district_update)
print("mukim_update:", mukim_update)
print("location_update:", location_update)


In [ ]:
# level 1,3,4

import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

state = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["state"]].drop_duplicates()['state'].str.lower().tolist()
district = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["district"]].drop_duplicates()['district'].str.lower().tolist()
mukim = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["mukim"]].dropna().drop_duplicates()['mukim'].str.lower().tolist()
location = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()

# find the state, district, mukim and location in the string
state_found = [s for s in state if s in str.lower()] # case len = 1 , case len = 0 , case len > 1
district_found = [d for d in district if d in str.lower()]
mukim_found = [m for m in mukim if m.lower() in str.lower()]
location_found = [l for l in location if l in str.lower()]  # Convert float to string

# 1)
if len(state_found) == 1:                                                                                                                 
    print("State found:", state_found[0])

    state_update=state_update+(state_found)

    state = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")
    
    #filter state_found
    state = state[state['state'].str.lower() == state_found[0]]

    #update mukim_found
    mukim = state[["mukim"]].dropna().drop_duplicates()['mukim'].str.lower().tolist()
    mukim_found = [x for x in [m for m in mukim if m.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]

    ####################################################################################################################


    # 4)
    if mukim_found:
        for mukim in mukim_found:
            # mukim name is not the same as district name
            if mukim != state_found[0]:
                print("Mukim name found not same as state name:", mukim)
                mukim_update=mukim_update+([mukim])
                # filter the mukim
                mukim_case1 = state[state['mukim'].str.lower() == mukim]
                # update location_found
                location = mukim_case1[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
                #location_found = [l for l in location if l in str.lower()]
                location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
                # 8) [148]
                if location_found:
                    for location in location_found:
                        if location != mukim_found[0]:
                            print(f"Location found for the mukim {mukim}:", location)
                            location_update=location_update+([location])
            # mukim name is the same as district name
            elif mukim == state_found[0]:
                print("Mukim name found same as state name")
                mukim_update=mukim_update+([mukim])
                # filter the mukim
                mukim_case2 = state[state['mukim'].str.lower() == mukim]
                # update location_found
                # 8) [148]
                location = mukim_case2[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
                #location_found = [l for l in location if l in str.lower()]

                location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]

                print(f"Location found for the mukim {mukim}:", location)
                location_update=location_update+([location])

##########################################################################################################

    # 5)    
    elif not mukim_found:
        print("No Mukim found for the state:", state_found[0])                    
        # update location_found
        location = state[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()
        #location_found = [l for l in location if l in str.lower()]
        location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
        # 10) [12510]
        if location_found:
            for location in location_found:
                if location != state_found[0]:
                    print(f"Location found for the state {state_found[0]}:", location)
                    location_update=location_update+([location])

##########################################################################################################
            
print()
print("state_update:", state_update)
print("district_update:", district_update)
print("mukim_update:", mukim_update)
print("location_update:", location_update)


In [ ]:
# level 1,4

import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

state = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["state"]].drop_duplicates()['state'].str.lower().tolist()
district = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["district"]].drop_duplicates()['district'].str.lower().tolist()
mukim = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["mukim"]].dropna().drop_duplicates()['mukim'].str.lower().tolist()
location = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")[["location"]].dropna().drop_duplicates()['location'].str.lower().tolist()

# find the state, district, mukim and location in the string
state_found = [s for s in state if s in str.lower()] # case len = 1 , case len = 0 , case len > 1
district_found = [d for d in district if d in str.lower()]
mukim_found = [m for m in mukim if m.lower() in str.lower()]
location_found = [l for l in location if l in str.lower()]  # Convert float to string

# 1)
if len(state_found) == 1:                                                                                                                 
    print("State found:", state_found[0])

    state_update=state_update+(state_found)

    state = pd.read_csv("malaysia-postcodes-location-mukim-district-state.csv")
    
    #filter state_found
    state = state[state['state'].str.lower() == state_found[0]]

    ####################################################################################################################

    #location_found = [l for l in location if l in str.lower()]
    location_found = [x for x in [l for l in location if l.lower() in str.lower()] if x in [x.strip() for x in str.lower().strip().split(',')]]
    # 8) [18]
    if location_found:
        for location in location_found:
            if location != state_found[0]:
                print(f"Location found for the state {state_found[0]}:", location)
                location_update=location_update+([location])

    ##########################################################################################################
            
print()
print("state_update:", state_update)
print("district_update:", district_update)
print("mukim_update:", mukim_update)
print("location_update:", location_update)


In [ ]:
state_update = list(set(state_update))
district_update = list(set(district_update))
mukim_update = list(set(mukim_update))
location_update = list(set(location_update))
print()
print("state_update:", state_update)
print("district_update:", district_update)
print("mukim_update:", mukim_update)
print("location_update:", location_update)

In [ ]:
from address_parser import extract_address_metadata


In [ ]:
input_string = "No. 4, Gerai Penjaja, Lorong Belakang 43300 Majlis Perbandaran Kajang Selangor"
extract_address_metadata(input_string)

In [ ]:
[x.strip() for x in str.lower().strip().split(',')]


In [ ]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3, "filter": {"$contains":"pulau pinang"}}) # 1)
similarity_score_threshold_retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}) # 2)
mmr_retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3 , 'lambda_mult': 0.9}) # 3)
#euclidean_retriever = vectorstore.as_retriever(search_type="euclidean", search_kwargs={"k": 5}) # 4)

# Establish EnsembleRetriever approach (combination of multiple retrievers)
ensemble_retriever = EnsembleRetriever(
    retrievers=[similarity_retriever,mmr_retriever],
    weights=[0.95,0.05],
    # retrieval_strategy="max",
    top_k=5
)

In [ ]:
# Create the RetrievalQA chain
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=similarity_retriever, return_source_documents=True)

In [ ]:
vectorstore.search("persiaran mayang pasir",k=3,search_type="similarity")

In [ ]:
# test the chain with a sample query
query = "persiaran mayang pasir, bayan lepas"
query = query.lower()
results = similarity_retriever.invoke(query)

for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

In [ ]:
results = vectorstore.similarity_search_with_score(
    "taman teratai",
    k=3,
    #filter={"$contains":"kedah"},
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [ ]:
results = vectorstore.similarity_search_by_vector(
    embedding=oembed.embed_query("kedah"), k=3
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")